El siguiente bloque descarga todos los archivos necesarios para la correcta ejecución del cuaderno.

In [12]:
%%capture
!mkdir v002
!mkdir images
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Agent.py' -O v002/Agent.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment.py' -O v002/Enviroment.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/__init__.py' -O v002/__init__.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/Enviroment_with_agents.py' -O v002/Enviroment_with_agents.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/v002/InOut_Simple_Laberinth.py' -O v002/InOut_Simple_Laberinth.py
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelNoTomato.bmp' -O images/PixelNoTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/PixelTomato.bmp' -O images/PixelTomato.bmp
!wget 'https://github.com/cgarcia-UCO/AgentSurvival/raw/main/images/face1_borders.bmp' -O images/face1_borders.bmp
!wget 'https://raw.githubusercontent.com/juanvs00/AgentSurvival/main/rules_to_clips_v3.py' -O rules_to_clips_v3.py

!pip install clipspy

from v002 import *
from rules_to_clips_v3 import *

En el siguiente bloque se deben escribir las reglas CLIPS y el nombre del agente.
El puzzle a resolver será que múltiples agentes peleen sobre un laberinto, gana el último con vida.

In [13]:
# TODO: Aquí debes escribir las reglas CLIPS para los agentes Runners
rules_run = []
rule1 = """
    (defrule giraIzqda
    (not (walls $? left $?))
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.turn_left()" "self.move_forward()"))
    )"""
rules_run.append(rule1)
rule2 = """
    (defrule haciaDelante
    (walls $? left $?)
    (not (walls $? front $?))
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )"""
rules_run.append(rule2)
rule3 = """
  (defrule asdf
  (available_object food type 1 ?y)
  (not (available_object agent ?a))
  ?h<-(pending_function_calls $?calls)
  =>
  (retract ?h)
  (assert (pending_function_calls ?y))
  )"""
rules_run.append(rule3)
rule4 = """
  (defrule comer
  (available_object food type 1 ?y)
  (not (available_object agent ?a))
  ?h<-(pending_function_calls $?calls)
  =>
  (retract ?h)
  (assert (pending_function_calls ?y))
  )"""
rules_run.append(rule4)

El siguiente bloque es un duplicado del anterior, en este puedes crear un diferente comportamiento para que haya dos tipos de agentes.
En el caso de que no quieras utilizar este segundo tipo de comportamiento, debes comentar aquellas líneas que tengan variables relativas a los agentes Punchers.

In [14]:
# TODO: Aquí debes escribir las reglas CLIPS para los agentes Punchers
rules_punch = []
rule1 = """
    (defrule giraDcha
    (not (walls $? right $?))
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.turn_right()" "self.move_forward()"))
    )"""
rules_punch.append(rule1)
rule2 = """
    (defrule haciaDelante
    (walls $? right $?)
    (not (walls $? front $?))
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.move_forward()"))
    )"""
rules_punch.append(rule2)
rule3 = """
    (defrule giraIzqda
    (walls $? right $?)
    (walls $? front $?)
    ?h<-(pending_function_calls $?calls)
    =>
    (retract ?h)
    (assert (pending_function_calls $?calls "self.turn_left()"))
    )"""
rules_punch.append(rule3)
rule4 = """
  (defrule golpear
  (available_object agent ?z)
  (not (available_object food type 1 ?z))
  ?h<-(pending_function_calls $?calls)
  =>
  (retract ?h)
  (assert (pending_function_calls ?z))
  )"""
rules_punch.append(rule4)

En el siguiente bloque se inicializan los tipos de agentes que vayas a utilizar.

In [15]:
runner = Agent_CLIPS(rules_run)
puncher = Agent_CLIPS(rules_punch)

El siguiente bloque son las funciones move de los agentes, no modificar.

In [16]:
def move_run(self, available_objects=None):

    # Actualizar el entorno CLIPS y obtener las interacciones del agente
    walls, objects, pos, ori = runner.agent_context(self)

    # Obtener las funciones y los objetos gracias al algoritmo
    functions, available_objects = algorithm(objects, runner.env, available_objects)

    # Ejecutar las funciones en pending_function_calls
    for j in functions:
        j = j.replace('"','')
        refresh_position(runner.env,j)
        eval(j)

def move_punch(self, available_objects=None):

    # Actualizar el entorno CLIPS y obtener las interacciones del agente
    walls, objetos, pos, ori = puncher.agent_context(self)

    # Obtener las funciones y los objetos gracias al algoritmo
    functions, available_objects = algorithm(objetos, puncher.env, available_objects)

    # Ejecutar las funciones en pending_function_calls
    for j in functions:
        j = j.replace('"','')
        refresh_position(puncher.env,j)
        eval(j)

El siguiente bloque es la ejecución del puzzle.
Puedes modificar el primer parámetro de la función Enviroment_with_agents, que corresponde con el tamaño del escenario.
También puedes modificar las variables del principio del bloque.

In [ ]:
num_agents = 8
hp = 150

reset_position(runner.env, runner.memory)
reset_position(puncher.env, puncher.memory)

lb1 = Enviroment_with_agents(10, max_moves_per_turn=7,
                                 plot_run='every epoch',
                                 move_protection=False, remove_walls_prob=0.5)
for i in range(int(num_agents/2)):
    lb1.create_agent('Runner' + str(i+1), move_run, life=hp)
    lb1.create_agent('Puncher' + str(i+1), move_punch, life=hp)

lb1.run()